In [ ]:
# HashTable class using chaining.
class HashTable:
    # Assigns all buckets with an empty list.
    def __init__(self, size=100):
        # initialize the hash table with empty bucket list entries.
        self.table = []
        self.length = 0
        for i in range(size):
            self.table.append([])
    def hash_key(self, key):
        return hash(key) % len(self.table)
    
    # Inserts a new item into the hash table.
    def add(self, key, value):
        # get the bucket where this item will go.
        bucket = self.hash_key(key)
        item = list([key,value])
        # Check if bucket has items
        if self.table[bucket]:
            found = False
            for _item_ in self.table[bucket]:
                if _item_[0] == key:
                    _item_[1] = value
                    found = True
            # If bucket does not have key, add item
            if not found:
                self.table[bucket].append(item)
                self.length +=1
        # If bucket has no items, add item
        else:
            self.table[bucket].append(item)
            self.length +=1
         
    # Searches for an item with matching key in the hash table.
    # Returns the item if found, or None if not found.
    def get(self, key):
        # get the bucket list where this key would be.
        bucket = self.hash_key(key)
        if self.table[bucket]:
            for _item_ in self.table[bucket]:
                if _item_[0] == key:
                    return _item_[1]    
        # If not found returns None
        print('Not found')
        return None

    # Removes an item with matching key from the hash table.
    def drop(self, key):
        # get the bucket list where this item will be removed from.
        bucket = self.hash_key(key)

        # remove the item from the bucket list if it is present.
        if self.table[bucket]:
            for index,_item_ in enumerate(self.table[bucket]):
                if _item_[0] == key:
                    self.table[bucket].pop(index)
                    print(f'{key} was removed')
                    self.length -=1
                    return
                
        # If not found return
        print(f'{key} not found')
        return
    def keys(self):
        keys = []
        for bucket in self.table:
            for item in bucket:
                keys.append(item[0])
        return keys
    def values(self):
        values = []
        for bucket in self.table:
            for item in bucket:
                values.append(item[1])
        return values
    def __str__(self):
        keys = self.keys()
        values = self.values()
        table_to_str = '{'
        for i in range(len(self.keys())):
            table_to_str += f"{repr(keys[i])}: "
            table_to_str += repr(values[i])
            if i != len(self.keys()) -1:
                table_to_str += ', ' +'\n'
            else:
                table_to_str += '}'
        return str(table_to_str)
    def __getitem__(self, key):
        return self.get(key)
    def __setitem__(self, key,value):
        self.add(key,value)
    def __len__(self):
        return self.length
    def __iter__(self):
        for key in self.keys():
            yield key, self.get(key)

In [ ]:
from collections import deque
from datetime import time
import csv

package_file = 'WGU_package_file.csv'
distance_file = 'WGUPS_distance_table.csv'


def get_vertices():
    vertices = []
    import csv
    with open(distance_file) as f:
        spam = csv.DictReader(f)
        for row in spam:
            vertices.append(row['vertex'])
    return vertices


def adjacency_matrix():
    vertices = []
    with open(distance_file) as f:
        spam = csv.DictReader(f)
        for row in spam:
            vertices.append(row['vertex'])
    dict_of_dict = {}
    for i in range(len(vertices)):
        single_dict = {}
        with open(distance_file) as f:
            spam = csv.DictReader(f)
            for row in spam:
                single_dict[int(row['vertex'])] = float(row[str(i)])
        dict_of_dict[i] = single_dict

    return dict_of_dict


def get_package_ids():
    ids = []
    with open(package_file) as f:
        spam = csv.DictReader(f)
        for row in spam:
            ids.append(row['package ID number'])

    return ids


def get_package_info(package_id):
    package_info = HashTable()
    with open(package_file) as f:
        spam = csv.DictReader(f)
        headers = spam.fieldnames
        for row in spam:
            if row['package ID number'] == package_id:
                for field in headers:
                    package_info.add(field, row[field])
                package_info.add('delivery status', 'at hub')

    return package_info




class Depot:
    def __init__(self):
        self.inventory = []
        self.priority_ready = []
        self.north_bound_ready = []
        self.north_bound_hold = []
        self.south_bound_priority = []
        self.south_bound_ready = []
        self.south_bound_hold = []

    def receive_packages(self):
        for item in get_package_ids():
            pkg = Package(item)
            pkg.at_hub = Time(8, 0).clock_time
            self.inventory.append(pkg)

    def route_priority(self):
        self.priority_ready = nearest_neighbor(0, self.priority_ready)

    def route_north_bound(self):
        self.north_bound_ready = nearest_neighbor(0, self.north_bound_ready)

    def route_north_bound_with_priority(self, priority):
        self.north_bound_ready = nearest_neighbor(priority, self.north_bound_ready)

    def route_south_bound(self):
        self.south_bound_ready = nearest_neighbor(0, self.south_bound_ready)

    def route_south_bound_with_priority(self, priority):
        self.south_bound_ready = nearest_neighbor(priority, self.south_bound_ready)

    def determine_truck_ready_hold(self, n_truck, s_truck):
        north_bound, south_bound = algos.map_direction()
        for pkg in self.inventory:
            if pkg.vertex in north_bound:
                if 'delayed' in pkg.info['Special Notes'].lower() and n_truck.time.time_delta < time(9,5):
                    self.north_bound_hold.append(pkg)
                elif 'wrong address' in pkg.info['Special Notes'].lower() and n_truck.time.time_delta < time(10,20):
                    self.north_bound_hold.append(pkg)
                else:
                    self.north_bound_ready.append(pkg)
            else:
                if 'delayed' in pkg.info['Special Notes'].lower() and s_truck.time.time_delta < time(9,5):
                    self.south_bound_hold.append(pkg)
                elif 'wrong address' in pkg.info['Special Notes'].lower() and s_truck.time.time_delta < time(10,20):
                    self.south_bound_hold.append(pkg)
                else:
                    self.south_bound_ready.append(pkg)

    def load_priority(self, truck):
        while self.priority_ready:
            pkg = self.priority_ready.pop(0)
            pkg.en_route = truck.time.clock_time
            truck.inventory.append(pkg)
            if len(truck.inventory) == truck.capacity:
                print('Truck is full')
                break

    def load_north_bound(self, truck):
        while self.north_bound_ready:
            pkg = self.north_bound_ready.pop(0)
            pkg.en_route = truck.time.clock_time
            truck.inventory.append(pkg)
            if len(truck.inventory) == truck.capacity:
                print('Truck is full')
                break
        if len(self.north_bound_ready) + len(self.north_bound_hold) > 0:
            truck.return_needed = True
        else:
            truck.return_needed = False

    def load_south_bound(self, truck):
        while self.south_bound_ready:
            pkg = self.south_bound_ready.pop(0)
            pkg.en_route = truck.time.clock_time
            truck.inventory.append(pkg)
            if len(truck.inventory) == truck.capacity:
                print('Truck is full')
                break
        if len(self.south_bound_ready) + len(self.south_bound_hold) > 0:
            truck.return_needed = True
        else:
            truck.return_needed = False

    def ready_held_packages(self,n_truck,s_truck):
        while self.north_bound_hold:
            pkg = self.north_bound_hold[0]
            if 'delayed' in pkg.info['Special Notes'].lower() and n_truck.time.time_delta < time(9, 5):
                continue
            elif 'wrong address' in pkg.info['Special Notes'].lower() and n_truck.time.time_delta < time(10, 20):
                continue
            pkg = self.north_bound_hold.pop(0)
            self.north_bound_ready.append(pkg)
        while self.south_bound_hold:
            pkg = self.south_bound_hold[0]
            if 'delayed' in pkg.info['Special Notes'].lower() and s_truck.time.time_delta < time(9, 5):
                continue
            elif 'wrong address' in pkg.info['Special Notes'].lower() and s_truck.time.time_delta < time(10, 20):
                continue
            pkg = self.south_bound_hold.pop(0)
            self.south_bound_ready.append(pkg)

    def place_package_on_hold(self, pkg_id):
        pass

    def special_case_packages(self, special_case_list):
        for pkg in self.inventory:
            if pkg.info['package ID number'] in special_case_list:
                pass

from datetime import time, timedelta, date, datetime


class Time:
    def __init__(self, hrs, minutes):
        self.time_delta = time(hrs, minutes, 0)
        self.clock_time = datetime.combine(date.today(), self.time_delta).strftime("%H:%M %p")

    def add(self, _minutes_):
        self.time_delta = (datetime.combine(date.today(), self.time_delta) + timedelta(minutes=_minutes_)).time()
        self.clock_time = datetime.combine(date.today(), self.time_delta).strftime("%H:%M %p")
        print(self.clock_time)

class Truck:
    def __init__(self):
        self.inventory = deque()
        self.time = time_delta.Time(8, 0)
        self.total_distance = 0
        self.speed = 18
        self.capacity = 16
        self.delivered = 0
        self.origin = 0
        self.return_needed = False

    def load(self, package):
        if len(self.inventory) == self.capacity:
            print('Truck is full')
            return
        self.inventory.append(package)

    def minutes_per_mile(self, distance):
        return round(60 / self.speed * distance, 0)

    def deliver(self):
        dict_dict = csv_reader.adjacency_matrix()
        if len(self.inventory) == 0:
            print('Empty')
            return
        elif self.time.time_delta > time(10, 20):
            self.fix_wrong_address_package()
        current_package = self.inventory.popleft()
        print(current_package)
        print(self.origin, '--->', current_package.info['vertex'])
        distance = float(dict_dict[current_package.vertex][self.origin])
        print('+', distance)
        self.time.add(self.minutes_per_mile(distance))
        self.total_distance += distance
        print(round(self.total_distance, 2))
        self.origin = current_package.vertex
        self.delivered += 1
        print(self.delivered, ' packages delivered')
        current_package.delivered = self.time.clock_time
        if len(self.inventory) == 0:
            self.return_to_hub()

    def return_to_hub(self):
        print('Returning to Hub')
        dict_dict = csv_reader.adjacency_matrix()
        distance = float(dict_dict[0][self.origin])
        self.time.add(self.minutes_per_mile(distance))
        self.total_distance += distance
        self.origin = 0
        print(round(self.total_distance, 2))
        print(self.time.clock_time)

    def get_total_distance(self):
        return round(self.total_distance, 2)

    def fix_wrong_address_package(self):
        for pkg in self.inventory:
            if 'wrong address' in pkg.info['Special Notes'].lower():
                pkg.info['vertex'] = 19

class Package:
    def __init__(self, package_id):
        self.info = get_package_info(package_id)
        self.vertex = int(self.info['vertex'])
        self.at_hub = None
        self.en_route = None
        self.delivered = None

    def __str__(self):
        return str('Package: ' + self.info['package ID number'])

In [ ]:
delivery_hub = Depot()
delivery_hub.receive_packages()

In [ ]:
pkg = delivery_hub.inventory[0]
add = pd.DataFrame()
add.loc[0,'package ID number'] = pkg.info['package ID number']
add.loc[0,'vertex'] = pkg.info['vertex']
add.loc[0,'delivery deadline'] = pkg.info['delivery deadline']
add.loc[0,'delivery status'] = pkg.info['delivery status']

In [ ]:
len(test)

In [ ]:
count = 0
len_t = len(test)
print(test)
while count < len_t:
    count += 1
    print(test[0])
    test.append(test.pop(0))
    print(test)

In [ ]:
test[0]

In [ ]:
test.pop(0)

In [39]:
from datetime import datetime
t1 = '10:21'
t2 = '1:00 AM'
t_s = datetime.strptime(t1, '%H:%M')
print(t_s.strftime("%H:%M %p"))

10:21 AM


In [32]:
datetime.strptime(t1, '%H:%M') > datetime.strptime(t2, '%H:%M %p')

True

In [40]:
datetime.strptime(t1, '%H:%M').time() > time(10,20)

True

In [35]:
print(datetime.strptime(t2, '%H:%M %p'))

1900-01-01 01:00:00


In [ ]:
test = []
print(len(test))

In [ ]:
import pandas as pd
_id = []
_vx = []
_dd = []
_sts = []

for pkg in delivery_hub.inventory:
    _id.append(pkg.info['package ID number'])
    _vx.append(pkg.info['vertex'])
    _dd.append(pkg.info['delivery deadline'])
    _sts.append(pkg.info['delivery status'])
df = pd.DataFrame({'package ID number': _id,'vertex':_vx,
                   'delivery deadline': _dd,'delivery status':_sts})

print(df.to_string(index=False))

In [ ]:
adj_matrix = df

In [ ]:
def nearest_neighbor_list(curr_vertex, adj_matrix, _list_):
    insert_index = 0
    while True:
        if insert_index == len(_list_)-1:
            return _list_
        nearest_dist = 2**32
        for i in _list_[insert_index:]:
            if adj_matrix.loc[i, curr_vertex] < nearest_dist:
                nearest_dist = adj_matrix.loc[i,curr_vertex]
                nearest_neighbor = i
        _list_.insert(insert_index,_list_.pop(_list_.index(nearest_neighbor)))
        curr_vertex = nearest_neighbor
        insert_index +=1
        #print(curr_vertex,'\n',insert_index)

In [ ]:
package_file = 'WGU_package_file.csv'
distance_file = 'WGUPS_distance_table.csv'


def get_vertices():
    vertices = []
    import csv
    with open(distance_file) as f:
        spam = csv.DictReader(f)
        for row in spam:
            vertices.append(row['vertex'])
    return vertices


def get_vertex_adjacency(vertex):
    vertex_adjacency_table = HashTable()
    print(vertex)
    if vertex == '0':
        print(vertex)
        return package.Package('1')
    with open(distance_file) as f:
        spam = csv.DictReader(f)
        for row in spam:
            vertex_adjacency_table.add(row['vertex'], row[vertex])
    return vertex_adjacency_table


def get_package_ids():
    ids = []
    with open(package_file) as f:
        spam = csv.DictReader(f)
        for row in spam:
            ids.append(row['package ID number'])

    return ids


def get_package_info(package_id):
    package_info = HashTable()
    with open(package_file) as f:
        spam = csv.DictReader(f)
        headers = spam.fieldnames
        for row in spam:
            if row['package ID number'] == package_id:
                for field in headers:
                    package_info.add(field, row[field])

    return package_info


In [ ]:
class Package:
    def __init__(self, package_id):
        self.info = get_package_info(package_id)
        
    def __str__(self):
        return str('Package: ' + self.info['package ID number'])

In [ ]:
class Depot:
    def __init__(self):
        self.inventory = []

    def receive_packages(self):
        for item in get_package_ids():
            self.inventory.append(Package(item))

    def determine_route(self):
        pass
        #self.inventory = nearest_neighbor(self.origin, self.inventory)


class Truck:
    def __init__(self):
        self.inventory = deque()
        self.total_time = 0
        self.total_distance = 0
        self.speed = 18

    def load(self, package):
        self.inventory.append(package)

    def minutes_per_mile(self, distance):
        return round(60 / self.speed * distance, 0)

    def deliver(self):
        if len(self.inventory) == 0:
            print('Empty')
            return
        current_package = self.inventory.popleft()
        print(current_package)
        print(self.origin, '--->', current_package.info['vertex'])
        distance = float(current_package.distance.get(self.origin))
        print(distance)
        self.total_time += self.minutes_per_mile(distance)
        self.total_distance += distance
        print(round(self.total_distance, 2))
        self.origin = current_package.info['vertex']

    def total_time(self):
        return self.total_time

    def total_distance(self):
        return round(self.total_distance, 2)

    def mph(self):
        return self.speed

In [ ]:
d_d = {}
for i in range(0,27):
    d_d[i] = dict(adj_matrix.loc[:, str(i)])


In [ ]:
distance_file = r'C:\Users\tssan\PycharmProjects\data_structures_algos_2\data\WGUPS_distance_table.csv'
vertices = []
import csv
with open(distance_file) as f:
    spam = csv.DictReader(f)
    for row in spam:
        vertices.append(row['vertex'])

In [ ]:
dict_of_dict = {}
for i in range(len(vertices)):
    single_dict = {}
    with open(distance_file) as f:
        spam = csv.DictReader(f)
        for row in spam:
            single_dict[int(row['vertex'])] = float(row[str(i)])
    dict_of_dict[i] = single_dict

In [ ]:
dict_of_dict[20][21]

In [ ]:
vertices = []
with open(distance_file) as f:
    spam = csv.DictReader(f)
    for row in spam:
        vertices.append(row['vertex'])
dict_of_dict = {}
for i in range(len(vertices)):
    single_dict = {}
    with open(distance_file) as f:
        spam = csv.DictReader(f)
        for row in spam:
            single_dict[int(row['vertex'])] = float(row[str(i)])
    dict_of_dict[i] = single_dict

In [ ]:
dict_of_dict[16][19]

In [ ]:
dict_of_dict[20][0]

In [ ]:
dict_of_dict[19][0]

In [ ]:
round(60 / 18 * 46, 0)

In [ ]:
from collections import deque
must_be = [0,2,3,4,5]
inv = deque([1,2,4,5,8,9])


while not all(pkg in inv for pkg in must_be):
    for pkg in must_be:
        if pkg not in inv:
            print(inv)
            inv.pop()
            print(inv)
            inv.appendleft(pkg)
            print(inv)
            print('next')

13,14 ,15,16,19,20

In [ ]:
prio = [6,20,21,21,4,17]

In [ ]:
def nearest_neighbor(curr_vertex, _list_):
    insert_index = 0
    better_list = []
    while _list_:
        if len(_list_) == 1:
            better_list.append(_list_.pop(0))
            return better_list
        nearest_dist = 2 ** 32
        for pkg in _list_[insert_index:]:
            if dict_of_dict[curr_vertex][pkg] < nearest_dist:
                nearest_dist = dict_of_dict[curr_vertex][pkg]
                nearest_package = pkg
                nearest_vertex = pkg
        _next_ = _list_.pop(_list_.index(nearest_package))
        better_list.append(_next_)
        curr_vertex = nearest_vertex

In [ ]:
nearest_neighbor(0,prio)

In [ ]:
sum_ = 0 
curr = 6
for i in nearest_neighbor(6,prio):
    sum_ += dict_of_dict[curr][i]
    curr=i
    print(sum_)

In [ ]:
dict_dict = dict_of_dict
maximum = -1
for i in dict_dict.keys():
    for j in dict_dict.keys():
        if dict_dict[i][j] > maximum:
            maximum = dict_dict[i][j]
            north = j
            south = i
minimum = 2 ** 32
mid = maximum / 2
skip = [north, south]
for v in dict_dict.keys():
    if v not in skip:
        sum_dist = dict_dict[v][north] + dict_dict[v][south]
        avg_dist = sum_dist / 2
        dif_dist = abs(avg_dist - mid)
        if dif_dist < minimum:
            minimum = dif_dist
            midpoint = v
skip.append(midpoint)
north_bound = [north]
south_bound = [south]
for v in dict_dict.keys():
    if v not in skip:
        if dict_dict[north][v] <= dict_dict[north][midpoint]:
            north_bound.append(v)
        else:
            south_bound.append(v)
if dict_dict[north][midpoint] < dict_dict[south][midpoint]:
    north_bound.append(midpoint)
else:
    south_bound.append(midpoint)

In [ ]:
north_bound

In [ ]:
south_bound

In [ ]:
d_d.keys()

In [ ]:
delivery_hub = Depot()
delivery_hub.receive_packages()
delivery_hub.determine_route()


In [ ]:
def nearest_neighbor_list(curr_vertex, adj_matrix, _list_):
    insert_index = 0
    while True:
        if insert_index == len(_list_)-1:
            return _list_
        nearest_dist = 2**32
        for i in _list_[insert_index:]:
            if adj_matrix.loc['Vertex'=='1', '0'] < nearest_dist:
                nearest_dist = adj_matrix.loc[int(i.info['vertex']),curr_vertex]
                next_item = i
                nearest_neighbor = i.info['vertex']
        _list_.insert(insert_index,_list_.pop(_list_.index(nearest_neighbor)))
        curr_vertex = nearest_neighbor
        insert_index +=1
        #print(curr_vertex,'\n',insert_index)

In [ ]:
adj_matrix

In [ ]:
nearest_neighbor_list('0',adj_matrix,delivery_hub.inventory)